El modelo se encuentra en el repositorio

In [1]:
!git clone https://github.com/dannasalazar11/Msc_thesis.git

Cloning into 'Msc_thesis'...
remote: Enumerating objects: 488, done.
remote: Counting objects: 100% (61/61), done.
remote: Compressing objects: 100% (61/61), done.
remote: Total 488 (delta 38), reused 0 (delta 0), pack-reused 427 (from 1)
Receiving objects: 100% (488/488), 50.49 MiB | 43.63 MiB/s, done.
Resolving deltas: 100% (315/315), done.


Creamos la señal arbitraria de EEG

In [2]:
import numpy as np

n_channels = 32
duration_sec = 10
sampling_rate = 250
n_samples = duration_sec * sampling_rate

X = np.random.randn(1000, n_channels, n_samples) # señal de EEG
y = np.eye(2)[np.random.randint(0, 2, size=1000)]

Cargamos el modelo y sus respectivos argumentos

In [3]:
import sys
sys.path.append('/kaggle/working/Msc_thesis')
from gmrrnet_adhd.models.GMRRNet import GMRRNet, NormalizedBinaryCrossentropy, RenyiMutualInformation
from tensorflow.keras.mixed_precision import set_global_policy
set_global_policy('mixed_float16')

from tensorflow.keras.utils import get_custom_objects
import tensorflow as tf

get_custom_objects().update({
    "RenyiMutualInformation": RenyiMutualInformation
})

model_name = 'GMRRNet'
model_args = {
    'num_kernels': 3,
    'nb_classes': 2,
    'Chans': n_channels,
    'Samples': n_samples,
    'norm_rate': 0.25,
    'alpha': 2,
    'num_heads': 2,
    'intermediate_dim':64
}


compile_args = {
    'loss':[NormalizedBinaryCrossentropy(name="NormalizedBinaryCrossentropy"),
        RenyiMutualInformation(C=tf.cast(19.0, tf.float64), name='MutualInfo')], 
    'optimizer': 'adam',
    'loss_weights' : [0.7, 0.3],
    'metrics' : [['binary_accuracy'], [None]]
}

model = GMRRNet(**model_args)

model.summary()

2025-07-22 14:33:07.558933: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753194787.933278      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753194788.039367      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
I0000 00:00:1753194811.692517      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1753194811.693248      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability:

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 32, 2500)  │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ cast (Cast)         │ (None, 32, 2500)  │          0 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape (Reshape)   │ (None, 2500, 32)  │          0 │ cast[0][0]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalization │ (None, 2500, 32)  │         64 │ reshape[0][0]     │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ transformer_encoder │ (None, 2500, 32)  │      8,544 │ layer_normalizat… │
│ (TransformerEncode… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 2500, 32)  │         64 │ transformer_enco… │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape_1 (Reshape) │ (None, 32, 2500,  │          0 │ layer_normalizat… │
│                     │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gaussian_layer_1    │ (None, 32, 32, 1) │          0 │ reshape_1[0][0]   │
│ (GaussianKernelLay… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gaussian_layer_2    │ (None, 32, 32, 1) │          0 │ reshape_1[0][0]   │
│ (GaussianKernelLay… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gaussian_layer_3    │ (None, 32, 32, 1) │          0 │ reshape_1[0][0]   │
│ (GaussianKernelLay… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 32, 32, 5) │         50 │ gaussian_layer_1… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 32, 32, 5) │         50 │ gaussian_layer_2… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 32, 32, 5) │         50 │ gaussian_layer_3… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 32, 32,    │          0 │ conv2d[0][0],     │
│ (Concatenate)       │ 15)               │            │ conv2d_1[0][0],   │
│                     │                   │            │ conv2d_2[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv2D_2 (Conv2D)   │ (None, 32, 32, 3) │        408 │ concatenate_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 32, 32, 3) │         12 │ Conv2D_2[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv2D_3 (Conv2D)   │ (None, 32, 32, 3) │         84 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 32, 32, 3) │         12 │ Conv2D_3[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 3072)      │          0 │ batch_normalizat

 Total params: 15,484 (60.48 KB)

 Trainable params: 15,472 (60.44 KB)

 Non-trainable params: 12 (48.00 B)

Lo compilamos y entrenamos con los datos

In [4]:
model.compile(**compile_args)

model.fit(X, [y,y], epochs=10)

Epoch 1/10


I0000 00:00:1753194824.925426      72 service.cc:148] XLA service 0x7f7fdc002c90 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1753194824.926885      72 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1753194824.926907      72 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1753194826.322130      72 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1753194842.291076      72 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


32/32 ━━━━━━━━━━━━━━━━━━━━ 40s 402ms/step - concatenated_entropies_loss: 0.7845 - loss: 0.6091 - out_activation_binary_accuracy: 0.4645 - out_activation_loss: 0.5337
Epoch 2/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 3s 85ms/step - concatenated_entropies_loss: 0.7845 - loss: 0.5887 - out_activation_binary_accuracy: 0.4952 - out_activation_loss: 0.5048
Epoch 3/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 3s 85ms/step - concatenated_entropies_loss: 0.7845 - loss: 0.5832 - out_activation_binary_accuracy: 0.5030 - out_activation_loss: 0.4970
Epoch 4/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 3s 85ms/step - concatenated_entropies_loss: 0.7845 - loss: 0.5704 - out_activation_binary_accuracy: 0.5213 - out_activation_loss: 0.4789
Epoch 5/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 3s 85ms/step - concatenated_entropies_loss: 0.7845 - loss: 0.5878 - out_activation_binary_accuracy: 0.4965 - out_activation_loss: 0.5033
Epoch 6/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 3s 85ms/step - concatenated_entropies_loss: 0.7845 - loss: 0.5952 - out_activation_binary_accura